# Мэтчинг

## Это всего лишь пример работы, которую нужно будет выполнить. Здесь разработана базовая модель для задачи мэтчинга и обучена на данных маркетплейса. Когда определимся с набором данных, я просто подсуну их этой модели и буду ее настраивать и тестировать.

В рамках проекта стоит задача определить пять наиболее близких по параметрам товаров на обезличенных данных от одного из крупнейших маркетплейсов страны.

**Описание данных**\
Для работы мы имеем 4 набра данных:
- base: Основной набор данных, который содержит всю имеющуюся базу товаров, где товары не размечены.
- train: Обучающий датасет, имеет разметку в виде наиболее схожего товара из перечня всей базы товаров.
- validation: Необходим для финального тестирования, имеет отдельно предоставленную разметку.
- validation_answer: Разметка для набора данных validation.

## 1. Установим необходимые компоненты и проведем импорты

In [1]:
!apt -q install libomp-dev
!pip -q install faiss-cpu --no-cache
!pip -q install faiss-gpu
!pip -q install optuna

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (788 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a14.0.0-1ubuntu1.1

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import silhouette_score
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA

import faiss
import time

import optuna

In [3]:
import requests
import os
from urllib.parse import urlencode
import zipfile

In [4]:
def downloader(size: str='small'):
    if size not in ['small', 'large']:
        raise Exception('Unknown Argument')
    elif size == 'small':
        public_key = 'https://disk.yandex.ru/d/YQElc_cNQQLSOw'
    else:
        public_key = 'https://disk.yandex.ru/d/BBEphK0EHSJ5Jw'

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open('/content/data.zip', 'wb') as f:
        f.write(download_response.content)

    zip_path = ('/content/data.zip')

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()

In [5]:
downloader('small')

## 2. Исследование и знакомство с данными

In [6]:
base = pd.read_csv("/content/base.csv", index_col="Id")

In [7]:
base.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
4207931-base,-43.946243,15.364378,17.515854,-132.31146,157.064420,-4.069252,-340.630860,-57.550140,128.398220,45.090958,...,-71.927170,30.711966,-90.190475,-24.931271,66.972534,106.346634,-44.270622,155.98834,-1074.464888,-25.066608
2710972-base,-73.004890,4.923342,-19.750746,-136.52908,99.907170,-70.709110,-567.401996,-128.890150,109.914986,201.472200,...,-109.044660,20.916021,-171.201390,-110.596844,67.730100,8.909615,-9.470253,133.29536,-545.897014,-72.913230
1371460-base,-85.565570,-0.493598,-48.374817,-157.98502,96.809510,-81.710210,-22.297688,79.768670,124.357086,105.715180,...,-58.821650,41.369606,-132.934500,-43.016839,67.871925,141.778240,69.048520,111.72038,-1111.038833,-23.087206
3438601-base,-105.564090,15.393871,-46.223934,-158.11488,79.514114,-48.944480,-93.713010,38.581398,123.397960,110.324326,...,-87.907290,-58.806870,-147.794800,-155.830237,68.974754,21.397510,126.098785,139.73320,-1282.707248,-74.527940
422798-base,-74.638880,11.315012,-40.204174,-161.76430,50.507114,-80.775560,-640.923467,65.225000,122.344940,191.465850,...,-30.002094,53.642930,-149.823230,176.921371,69.473280,-43.395180,-58.947716,133.84064,-1074.464888,-1.164146


In [9]:
train = pd.read_csv("./train.csv", index_col="Id")

In [10]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
109249-query,-24.021454,3.122524,-80.947525,-112.329994,191.090180,-66.903130,-759.626065,-75.284454,120.55149,131.131700,...,-24.601670,-167.760770,133.678516,68.184600,26.317545,11.938202,148.549320,-778.563381,-46.877750,66971-base
34137-query,-82.033580,8.115866,-8.793022,-182.972100,56.645336,-52.597610,-55.720337,130.059250,129.38335,76.202880,...,54.448433,-120.894806,-12.292085,66.608116,-27.997612,10.091335,95.809265,-1022.691531,-88.564705,1433819-base
136121-query,-75.719640,-0.223386,-86.186130,-162.064060,114.320114,-53.394600,-117.261013,-24.857851,124.80780,112.190155,...,-5.609123,-93.029880,-80.997871,63.733383,11.378683,62.932007,130.975390,-1074.464888,-74.861176,290133-base
105191-query,-56.580620,5.093593,-46.943110,-149.039120,112.436430,-76.820510,-324.995645,-32.833107,119.47865,120.074790,...,21.624313,-158.880370,179.597294,69.891360,-33.804955,233.914610,122.868546,-1074.464888,-93.775375,1270048-base
63983-query,-52.725650,9.027046,-92.829650,-113.111010,134.124970,-42.423073,-759.626065,8.261169,119.49023,172.365360,...,13.807772,-208.650040,41.742014,66.522420,41.362930,162.723050,111.261310,-151.162805,-33.831450,168591-base


In [21]:
scaler = RobustScaler()
scaler.fit(base)

RobustScaler()

## 3. Бейзлайн

In [29]:
base_scale = scaler.transform(base)

In [30]:
kmeans = KMeans(n_clusters=10, init="k-means++", max_iter=300, random_state=0)
kmeans.fit(base_scale)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10, random_state=0)

In [31]:
X = train.drop("Target", axis=1)
X_normalize = scaler.transform(X)

In [32]:
def top_func(data_norm, data, base_data):

  train_cluster_pred = kmeans.predict(data_norm)

  index_mapping = {i: idx for i, idx in enumerate(base.index)}

  cluster_index = {cluster: np.where(kmeans.labels_ == cluster)[0] for cluster in np.unique(kmeans.labels_)}

  accuracy = 0
  for cluster in tqdm(np.unique(train_cluster_pred), desc="Processing clusters"):
      base_cluster_index = cluster_index[cluster]
      base_cluster = base_data[base_cluster_index]
      train_cluster = data_norm[train_cluster_pred == cluster]
      true_labels_cluster = data[train_cluster_pred == cluster]['Target']

      neighbors = NearestNeighbors(n_neighbors=5)
      neighbors.fit(base_cluster)

      _, index = neighbors.kneighbors(train_cluster)

      for target, idx in zip(true_labels_cluster.values.tolist(), index):
          text_index = [index_mapping[base_cluster_index[i]] for i in idx]
          accuracy += int(target in text_index)

  general_accuracy = 100 * accuracy / len(train)
  print(f"Total Accuracy: {general_accuracy:.2f}%")


In [33]:
top_func(X_normalize, train, base_scale)

Processing clusters: 100%|██████████| 10/10 [00:19<00:00,  1.96s/it]

Total Accuracy: 60.80%


## 4. FAISS

Faiss - это инструмент, разработанный Facebook для быстрого поиска ближайших соседей в больших наборах данных.

Использование Faiss включает:
- подготовку данных  
- создание специального "индекса" для поиска  
- поиск ближайших соседей

In [39]:
sample_ = train.sample(n=1_000, random_state=0)
sample_x = sample_.drop("Target", axis=1)
sample_x_normalize = scaler.transform(sample_x)

base_data = base_scale.astype(np.float32)
train_data = sample_x_normalize.astype(np.float32)
train_labels = sample_['Target']

In [40]:
def index_func(data,
               n_clusters=None,
               use_pca=False,
               pca_dimensions=None,
               metric=faiss.METRIC_L2):
    d = data.shape[1]

    if use_pca and pca_dimensions:
        pca_matrix = faiss.PCAMatrix(d, pca_dimensions, eigen_power=-0.5)
        pca_matrix.train(data)
        data = pca_matrix.apply_py(data)
        d = pca_dimensions

    if n_clusters:
        quantizer = faiss.IndexFlatL2(d)
        index = faiss.IndexIVFFlat(quantizer, d, n_clusters, metric)
        index.train(data)
    else:
        index = faiss.IndexFlatL2(d)

    index.add(data)
    return index, (pca_matrix if use_pca and pca_dimensions else None)

In [41]:
def search_func(index, query_data, k=5, pca_matrix=None):
    start_time = time.time()
    if pca_matrix:
        query_data = pca_matrix.apply_py(query_data)
    distances, indices = index.search(query_data, k)
    print(f"Поиск занял {time.time() - start_time:.2f} секунд")
    return distances, indices

In [42]:
def accuracy_func(indices, train_labels, index_to_label):
    matches = sum(label in [index_to_label[idx] for idx in neighbors] for label, neighbors in zip(train_labels, indices))
    accuracy = 100 * matches / len(train_labels)
    return accuracy

In [43]:
def match_func(base_data, train_data, train_labels,
                     n_clusters=None,
                     use_pca=False,
                     pca_dimensions=None,
                     k=5, nprobe=2,
                     metric=faiss.METRIC_L2):

    index, pca_matrix = index_func(base_data,
                                   n_clusters=n_clusters,
                                   use_pca=use_pca,
                                   pca_dimensions=pca_dimensions,
                                   metric=metric)

    index.nprobe = nprobe

    _, indices = search_func(index, train_data, k=k, pca_matrix=pca_matrix)

    index_to_label = {idx: label for idx, label in enumerate(base.index)}

    accuracy = accuracy_func(indices, train_labels, index_to_label)
    return accuracy


In [44]:
def params_func(trial):

    n_clusters = trial.suggest_int('n_clusters', 10, 500, step=10)

    pca_dimensions_option = trial.suggest_categorical('pca_dimensions_option', [True, False])
    pca_dimensions = trial.suggest_int('pca_dimensions', 27, 70) if pca_dimensions_option else None

    nprobe = trial.suggest_int('nprobe', 1, 10)

    base_data_c = np.ascontiguousarray(base_data)
    train_data_c = np.ascontiguousarray(train_data)

    accuracy = match_func(base_data_c, train_data_c, train_labels,
                          n_clusters=n_clusters,
                          use_pca=pca_dimensions_option,
                          pca_dimensions=pca_dimensions,
                          k=5, nprobe=nprobe)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(params_func, n_trials= 25)

print("Лучшая метрика:", study.best_value)
print("Лучшие параметры:", study.best_params)

[I 2024-05-07 09:45:12,446] A new study created in memory with name: no-name-84a2102d-75cc-4fa5-aba9-c41afb6546ec
[I 2024-05-07 09:45:14,251] Trial 0 finished with value: 73.5 and parameters: {'n_clusters': 150, 'pca_dimensions_option': True, 'pca_dimensions': 53, 'nprobe': 9}. Best is trial 0 with value: 73.5.


Поиск занял 0.60 секунд


[I 2024-05-07 09:45:15,334] Trial 1 finished with value: 63.0 and parameters: {'n_clusters': 130, 'pca_dimensions_option': True, 'pca_dimensions': 31, 'nprobe': 5}. Best is trial 0 with value: 73.5.


Поиск занял 0.27 секунд


[I 2024-05-07 09:45:17,268] Trial 2 finished with value: 75.0 and parameters: {'n_clusters': 80, 'pca_dimensions_option': True, 'pca_dimensions': 62, 'nprobe': 7}. Best is trial 2 with value: 75.0.


...

Поиск занял 0.57 секунд


[I 2024-05-07 09:46:47,573] Trial 23 finished with value: 72.0 and parameters: {'n_clusters': 100, 'pca_dimensions_option': True, 'pca_dimensions': 55, 'nprobe': 4}. Best is trial 16 with value: 76.4.


Поиск занял 0.45 секунд


[I 2024-05-07 09:46:49,976] Trial 24 finished with value: 74.2 and parameters: {'n_clusters': 40, 'pca_dimensions_option': True, 'pca_dimensions': 59, 'nprobe': 6}. Best is trial 16 with value: 76.4.


Поиск занял 1.69 секунд
Лучшая метрика: 76.4
Лучшие параметры: {'n_clusters': 20, 'pca_dimensions_option': True, 'pca_dimensions': 62, 'nprobe': 7}


Лучшая метрика: 76.4\
Лучшие параметры: {'n_clusters': 20, 'pca_dimensions_option': True, 'pca_dimensions': 62, 'nprobe': 7}